# 如何用Logger做实验？

这一节主要介绍FLGo中的日志记录模块。

## 1 使用Logger进行实验记录

FLGo中的日志记录器Logger用于观察训练过程中的实时结果，并记录到字典（logger.output）中，存在相应任务目录的record文件夹中。 Logger主要通过提供以下3个接口函数，来帮助用户达到实验目的。

* `initialize`：预留的初始化方法；

* `log_once`：每隔一定通信轮次，被调用一次，来记录\输出实时结果到变量`self.output`（类型为`collections.defaultdict(list)`）中；

* `organize_output`：在将`self.output`保存成.json文件之前，对特定内容进行组织，使得相应内容可以被直接用于绘制等目的；


由于这3个接口函数中，大多数时候都只用得到`log_once`函数，这里也仅对该函数的用法进行适当介绍。FLGo中所有的`Logger`都需要继承自`flgo.experiment.logger.BasicLogger`。当不指定`Logger`时，默认使用的`Logger`为`flgo.experiment.logger.SimpleLogger`，其代码实现如下：

In [3]:
from flgo.experiment.logger import BasicLogger
import numpy as np
import flgo.simulator.base as ss

class SimpleLogger(BasicLogger):
    def initialize(self):
        """在输出output中记录各用户的本地数据量，用户使用self.participants属性访问，服务器使用self.coordinator属性访问。self.output的默认键值为空列表"""
        for c in self.participants:
            self.output['client_datavol'].append(len(c.train_data))

    def log_once(self, *args, **kwargs):
        # 服务器（coordinator）使用test方法测试全局模型的测试集性能，并记录至output中
        test_metric = self.coordinator.test()
        for met_name, met_val in test_metric.items():
            self.output['test_' + met_name].append(met_val)
        # 服务器（coordinator）使用global_test方法测试全局模型的用户本地验证集性能分布，并记录至output中
        val_metrics = self.coordinator.global_test('val')
        local_data_vols = [c.datavol for c in self.participants]
        total_data_vol = sum(local_data_vols)
        for met_name, met_val in val_metrics.items():
            self.output['val_'+met_name+'_dist'].append(met_val)
            self.output['val_' + met_name].append(1.0 * sum([client_vol * client_met for client_vol, client_met in zip(local_data_vols, met_val)]) / total_data_vol)
            self.output['mean_val_' + met_name].append(np.mean(met_val))
            self.output['std_val_' + met_name].append(np.std(met_val))
        # 将当前output中信息输出至控制台
        self.show_current_output()

默认的`SimpleLogger`每一轮测试模型的测试集性能和验证集性能，并记录。最后`output`字典在`runner`训练结束之后会被存成.json文件。

### 定制自己的Logger

这里通过下面的例子演示如何定制自己的`Logger`。这个`Logger`每一轮仅测试模型的测试集性能，并记录测试集指标最佳的全局模型为最优全局模型。最后用最优全局模型来测试本地用户数据的验证集性能分布，记录验证集性能分布中最佳30%用户和最差30%用户的平均性能。为了实现这些功能，实现`Logger`如下：

In [31]:
from flgo.experiment.logger import BasicLogger
import collections
import numpy as np
import copy

class MyLogger(BasicLogger):
    def initialize(self, *args, **kwargs):
        self.optimal_model = copy.deepcopy(self.coordinator.model)
        self.optimal_test_loss = 9999
    
    def log_once(self):
        # 测模型测试集指标
        test_metric = self.coordinator.test()
        for met_name, met_val in test_metric.items():
            self.output['test_' + met_name].append(met_val)
        # 检测当前模型是否为最优模型
        if test_metric['loss']<self.optimal_test_loss:
            self.optimal_test_loss = test_metric['loss']
            self.optimal_model.load_state_dict(self.coordinator.model.state_dict())
        self.show_current_output()
    
    def organize_output(self):
        super().organize_output()
        # 测所有用户验证集指标
        all_metrics = collections.defaultdict(list)
        for c in self.participants:
            client_metrics = c.test(self.optimal_model, 'val')
            for met_name, met_val in client_metrics.items():
                all_metrics[met_name].append(met_val)
        for met_name, metval in all_metrics.items():
            self.output[met_name] = metval
        # 计算最佳\最差30%用户验证集指标
        met_name = 'loss'
        all_val_losses = sorted(all_metrics[met_name])
        k1 = int(0.3*len(self.participants))
        k2 = int(0.7*len(self.participants))
        self.output['worst_30_val_loss'] = 1.0*sum(all_val_losses[k2:])/k1
        self.output['best_30_val_loss']  = 1.0*sum(all_val_losses[:k1])/k1

下面验证所实现的`MyLogger`的效果：

In [ ]:
import flgo
import flgo.algorithm.fedavg as fedavg
import flgo.algorithm.qfedavg as qfedavg
import os

task = './test_synthetic'
config = {'benchmark':{'name':'flgo.benchmark.synthetic_regression', 'para':{'alpha':0.5, 'beta':0.5, 'num_clients':30}}}
if not os.path.exists(task): flgo.gen_task(config, task_path = task)

op = {'num_rounds':30, 'num_epochs':1, 'batch_size':8, 'learning_rate':0.1, 'proportion':1.0 ,'gpu':0, 'algo_para':0.1}
fedavg_runner = flgo.init(task, fedavg, option = op, Logger=MyLogger)
qffl_runner = flgo.init(task, qfedavg, option=op, Logger=MyLogger)
fedavg_runner.run()
qffl_runner.run()

In [41]:
import flgo.experiment.analyzer as al
records = al.Selector({'task':task, 'header':['fedavg', 'qfedavg_q0.1',], 'filter':{'R':30, 'E':1, 'B':8, 'LR':0.1,'P':1.0}}).records[task]
for rec in records:
    wl = rec.data['worst_30_val_loss']
    bl = rec.data['best_30_val_loss']
    print('{}:(Worst is {}, Best is {})'.format(rec.data['option']['algorithm'], wl, bl))

fedavg:(Worst is 1.539149112171597, Best is 0.1532415940115849)
qfedavg:(Worst is 1.5319806469811335, Best is 0.4070415910747316)


可以看到记录中相应关键字的内容已被保存，且可以被读取。其中qfedavg的最差30%用户性能略好于fedavg，但最佳30%用户性能若弱于fedavg。